# map data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import warnings                             
warnings.filterwarnings("ignore")

In [2]:
%%time
train = pd.read_csv('data2/train.csv')

CPU times: user 49.1 s, sys: 32.7 s, total: 1min 21s
Wall time: 2min 15s


In [3]:
%%time
test = pd.read_csv('data2/test.csv')

CPU times: user 20.5 s, sys: 10 s, total: 30.5 s
Wall time: 57.1 s


In [206]:
df_train = pd.DataFrame(train.game_id.unique(), columns=['game_id'])
df_train.index = df_train.game_id
df_train = df_train.drop(['game_id'], axis = 1)

In [196]:
df_test = pd.DataFrame(test.game_id.unique(), columns=['game_id'])
df_test.index = df_test.game_id
df_test = df_test.drop(['game_id'], axis = 1)

In [207]:
%%time
# 처음 기록 된 카메라 좌표를 기록

df_train_p0 = train[(train.event=='Camera')&(train.player==0)]
df_train_p0 = df_train_p0[df_train_p0.shift(1).game_id!=df_train_p0.game_id] # 쉬프트를 이용하여 각 게임의 첫번째 데이터 찾기
df_train_p0 = df_train_p0.iloc[:, [0,6]].rename({'event_contents':'player0_starting'}, axis = 1)
df_train_p0.index = df_train_p0['game_id']
df_train_p0 = df_train_p0.drop(['game_id'], axis=1)
df_train = pd.merge(df_train, df_train_p0, on='game_id', how='left')
del df_train_p0

df_train_p1 = train[(train.event=='Camera')&(train.player==1)]
df_train_p1 = df_train_p1[df_train_p1.shift(1).game_id!=df_train_p1.game_id]
df_train_p1 = df_train_p1.iloc[:, [0,6]].rename({'event_contents':'player1_starting'}, axis = 1)
df_train_p1.index = df_train_p1['game_id']
df_train_p1 = df_train_p1.drop(['game_id'], axis=1)
df_train = pd.merge(df_train, df_train_p1, on='game_id', how='left')
del df_train_p1

CPU times: user 14.1 s, sys: 26.7 s, total: 40.9 s
Wall time: 2min 52s


In [200]:
%%time
# 처음 기록 된 카메라 좌표를 기록

df_test_p0 = test[(test.event=='Camera')&(test.player==0)]
df_test_p0 = df_test_p0[df_test_p0.shift(1).game_id!=df_test_p0.game_id] # 쉬프트를 이용하여 각 게임의 첫번째 데이터 찾기
df_test_p0 = df_test_p0.iloc[:, [0,5]].rename({'event_contents':'player0_starting'}, axis = 1)
df_test_p0.index = df_test_p0['game_id']
df_test_p0 = df_test_p0.drop(['game_id'], axis=1)
df_test = pd.merge(df_test, df_test_p0, on='game_id', how='left')
del df_test_p0

df_test_p1 = test[(test.event=='Camera')&(test.player==1)]
df_test_p1 = df_test_p1[df_test_p1.shift(1).game_id!=df_test_p1.game_id]
df_test_p1 = df_test_p1.iloc[:, [0,5]].rename({'event_contents':'player1_starting'}, axis = 1)
df_test_p1.index = df_test_p1['game_id']
df_test_p1 = df_test_p1.drop(['game_id'], axis=1)
df_test = pd.merge(df_test, df_test_p1, on='game_id', how='left')
del df_test_p1

CPU times: user 4.62 s, sys: 1.12 s, total: 5.73 s
Wall time: 9.5 s


In [208]:
# x, y 값으로 분리

df_train['player0_starting'] = df_train.player0_starting.str.split('(').str[1]
df_train['player0_starting'] = df_train.player0_starting.str.split(')').str[0]
split_xy = df_train.player0_starting.str.split(',')
df_train['player0_x'] = split_xy.str[0].astype('float')
df_train['player0_y'] = split_xy.str[1].astype('float')
del split_xy

df_train['player1_starting'] = df_train.player1_starting.str.split('(').str[1]
df_train['player1_starting'] = df_train.player1_starting.str.split(')').str[0]
split_xy = df_train.player1_starting.str.split(',')
df_train['player1_x'] = split_xy.str[0].astype('float')
df_train['player1_y'] = split_xy.str[1].astype('float')
del split_xy

In [201]:
# x, y 값으로 분리

df_test['player0_starting'] = df_test.player0_starting.str.split('(').str[1]
df_test['player0_starting'] = df_test.player0_starting.str.split(')').str[0]
split_xy = df_test.player0_starting.str.split(',')
df_test['player0_x'] = split_xy.str[0].astype('float')
df_test['player0_y'] = split_xy.str[1].astype('float')
del split_xy

df_test['player1_starting'] = df_test.player1_starting.str.split('(').str[1]
df_test['player1_starting'] = df_test.player1_starting.str.split(')').str[0]
split_xy = df_test.player1_starting.str.split(',')
df_test['player1_x'] = split_xy.str[0].astype('float')
df_test['player1_y'] = split_xy.str[1].astype('float')
del split_xy

In [209]:
# 플레이어의 x,y 좌표를 하나로 모음

location_p0 = df_train.loc[:, ['player0_x', 'player0_y']]
location_p0 = location_p0.rename({'player0_x':'location_x', 'player0_y':'location_y'}, axis=1)

location_p1 = df_train.loc[:, ['player1_x', 'player1_y']]
location_p1 = location_p1.rename({'player1_x':'location_x', 'player1_y':'location_y'}, axis=1)
location_p1.index += location_p0.index[-1]+1

location = pd.concat([location_p0, location_p1])
location = location.dropna()
del location_p0, location_p1

In [202]:
# # 플레이어의 x,y 좌표를 하나로 모음

# location_p0 = df_test.loc[:, ['player0_x', 'player0_y']]
# location_p0 = location_p0.rename({'player0_x':'location_x', 'player0_y':'location_y'}, axis=1)

# location_p1 = df_test.loc[:, ['player1_x', 'player1_y']]
# location_p1 = location_p1.rename({'player1_x':'location_x', 'player1_y':'location_y'}, axis=1)
# location_p1.index += location_p0.index[-1]+1

# location = pd.concat([location_p0, location_p1])
# location = location.dropna()
# del location_p0, location_p1

In [153]:
# # 모든 포인트 시각화
# sns.lmplot('location_x', 'location_y', data = location, fit_reg=False)
# plt.title('starting point')
# plt.show()

In [154]:
# # 스타팅 컬럼을 카운팅을 해보면 15개의 포인트가 많음
# # 15개의 스타팅포인트 존재
# df_train.player0_starting.value_counts().head(20)

In [210]:
# kmeans를 이용하여 15개로 클러스터링
kmeans_clst = KMeans(n_clusters=15).fit(location)
location['starting'] = kmeans_clst.labels_+1

In [211]:
# location.head()

In [212]:
# # 클러스트링한 결과를 시각화
# sns.lmplot('location_x', 'location_y', data = location, fit_reg=False, hue="starting")
# plt.title('starting point')
# plt.show()

# # 초반 진행 화면이 누락 된 게임은 엉뚱하게 분류 됨

In [213]:
# kmeans로 찾은 15개의 포인트에서 각 데이터들의 거리 계산
for cluster in range(15):
    point = location[location.starting==cluster+1]
    loc = point.loc[:,['location_x', 'location_y']]
    del point
    loc['center_x'] = kmeans_clst.cluster_centers_[cluster][0]
    loc['center_y'] = kmeans_clst.cluster_centers_[cluster][1]
    distance = np.sqrt(np. square(loc.location_x - loc.center_x) + np.square(loc.location_y - loc.center_y))
    location.loc[loc.index, 'distance'] = distance
    del loc

In [214]:
# 일정 거리(5)이상 떨어진 데이터는 starting을 0으로 지정
idx = location[location.distance>5].index
location.loc[idx, 'starting'] = 0
del idx

In [215]:
# # 시각화를 통해 스타팅을 모르는 게임은 분홍색(0)으로 나타며 15개의 스타팅 포인트를 확인 할 수 있음.
# sns.lmplot('location_x', 'location_y', data = location, fit_reg=False, hue="starting")
# plt.title('starting point')
# plt.show()

In [216]:
# df_train.head()

In [217]:
df_train_temp = df_train.reset_index()[['game_id', 'player0_starting', 'player1_starting']]

location['xy'] = location.apply(lambda x: str(x.location_x) + ', ' + str(x.location_y), axis='columns')

location_temp = location.reset_index()[['game_id', 'xy', 'starting']]

ttt = df_train_temp.merge(location_temp, how='left', on='game_id' )

location.index -= df_train.index[-1]+1

location_temp_2 = location.reset_index()[['game_id', 'xy', 'starting']]

ttt_2 = df_train_temp.merge(location_temp_2, how='left', on='game_id' )

ttt_3 = ttt.merge(ttt_2[['game_id','starting']], how='left', on='game_id')

ttt_3 = ttt_3.drop(['player0_starting','player1_starting','xy'], axis='columns')

ttt_3 = ttt_3.set_index('game_id')

ttt_3 = ttt_3.rename({'starting_x':'player0_starting', 'starting_y':'player1_starting'}, axis='columns')

ttt_3 = ttt_3.fillna(0)

df_train = ttt_3

In [242]:
df_test.head()

,player0_starting,player1_starting,player0_x,player0_y,player1_x,player1_y
game_id,,,,,,
38872,"22.25, 81.5078125","120.25, 153.83984375",22.25,81.507812,120.25,153.839844
38873,"24.75, 128.0078125","151.25, 20.5078125",24.75,128.007812,151.25,20.507812
38874,"151.25, 20.5078125","24.75, 128.0078125",151.25,20.507812,24.75,128.007812
38875,"152.25, 142.0078125","31.75, 22.5078125",152.25,142.007812,31.75,22.507812
38876,"135.25, 31.5078125","22.25, 81.5078125",135.25,31.507812,22.25,81.507812


안돼!

In [217]:
df_test_temp = df_test.reset_index()[['game_id', 'player0_starting', 'player1_starting']]

location['xy'] = location.apply(lambda x: str(x.location_x) + ', ' + str(x.location_y), axis='columns')

location_temp = location.reset_index()[['game_id', 'xy', 'starting']]

ttt = df_test_temp.merge(location_temp, how='left', on='game_id' )

location.index -= df_test.index[-1]+1

location_temp_2 = location.reset_index()[['game_id', 'xy', 'starting']]

ttt_2 = df_test_temp.merge(location_temp_2, how='left', on='game_id' )

ttt_3 = ttt.merge(ttt_2[['game_id','starting']], how='left', on='game_id')

ttt_3 = ttt_3.drop(['player0_starting','player1_starting','xy'], axis='columns')

ttt_3 = ttt_3.set_index('game_id')

ttt_3 = ttt_3.rename({'starting_x':'player0_starting', 'starting_y':'player1_starting'}, axis='columns')

ttt_3 = ttt_3.fillna(0)

df_test = ttt_3

In [232]:
# 스타팅 포인트를 이용하여 맵 분류
map_list = []
for point in range(1,16):
    couple = df_train[df_train.player0_starting == point].player1_starting.value_counts()
    if couple[couple.index[1]]<100:
        map_list.append([point, couple.index[0], 999])
    else:
        map_list.append([point, couple.index[0], couple.index[1]])
map_list = np.sort(map_list, axis = 1)
map_list = np.unique(map_list, axis = 0)

In [233]:
# 6개의 2인용 맵과 1개의 3인용 맵이 존재

# TMI: 스타크래프트2 공허의유산 래더에서 3인용 맵이 쓰인적은 '까탈레나' 딱 한 번.
# 이를 통해 2017년 7월20일 ~ 2017년 11월16일 사이에 진행 된 게임 정보라는 것을 알 수 있음.
# 나머지 6개의 맵은 '어비설리프', '어센션투아이어', '애컬라이트', '인터로퍼', '오딧세이', '메크디포'
# 근데 결과 예측하는데 맵 이름 상관 없음
map_list

array([[  1.,   2., 999.],
       [  3.,   4., 999.],
       [  5.,   9., 999.],
       [  6.,   8.,  11.],
       [  7.,  15., 999.],
       [ 10.,  14., 999.],
       [ 12.,  13., 999.]])

In [234]:
# 스타팅을 모르는 게임 수 확인.
len(df_train[(df_train.player0_starting == 0)|(df_train.player1_starting == 0)])

258

In [235]:
# map_list와 상대편 위치 정보를 이용하여 모르는 스타팅 찾기
for m in map_list:
    idx = df_train[(df_train.player0_starting == 0)&((df_train.player1_starting == m[0])|(df_train.player1_starting == m[2]))].index
    df_train.loc[idx, 'player0_starting'] = m[1]
    del idx
    idx = df_train[(df_train.player0_starting == 0)&((df_train.player1_starting == m[1])|(df_train.player1_starting == m[2]))].index
    df_train.loc[idx, 'player0_starting'] = m[0]
    del idx
    
    idx = df_train[(df_train.player1_starting == 0)&((df_train.player0_starting == m[0])|(df_train.player0_starting == m[2]))].index
    df_train.loc[idx, 'player1_starting'] = m[1]
    del idx
    idx = df_train[(df_train.player1_starting == 0)&((df_train.player0_starting == m[1])|(df_train.player0_starting == m[2]))].index
    df_train.loc[idx, 'player1_starting'] = m[0]
    del idx

In [236]:
# 모든 게임의 스타팅포인트를 찾음
df_train[(df_train.player0_starting == 0)|(df_train.player1_starting == 0)].head()

,player0_starting,player1_starting
game_id,,


In [237]:
# 맵 컬럼 추가
for map_num, m in enumerate(map_list):
    idx = df_train[(df_train.player0_starting == m[0])|(df_train.player0_starting == m[1])|(df_train.player0_starting == m[2])].index
    df_train.loc[idx, 'map'] = map_num
del idx, map_list

In [239]:
df_train.head()

,player0_starting,player1_starting,map
game_id,,,
0,2.0,1.0,0.0
1,3.0,4.0,1.0
2,14.0,10.0,5.0
3,13.0,12.0,6.0
4,5.0,9.0,2.0


In [237]:
# 맵 컬럼 추가
for map_num, m in enumerate(map_list):
    idx = df_train[(df_train.player0_starting == m[0])|(df_train.player0_starting == m[1])|(df_train.player0_starting == m[2])].index
    df_train.loc[idx, 'map'] = map_num
del idx, map_list

In [238]:
# 스타팅포인트, 맵 클러스터링 끝
df_train.head()

,player0_starting,player1_starting,map
game_id,,,
0,2.0,1.0,0.0
1,3.0,4.0,1.0
2,14.0,10.0,5.0
3,13.0,12.0,6.0
4,5.0,9.0,2.0


플레이어 0, 1 합쳐서 보지 뭐

In [19]:
df_train.head()

,player0_starting,player1_starting,player0_x,player0_y,player1_x,player1_y
game_id,,,,,,
0,"145.25, 21.5078125","22.75, 147.0078125",145.25,21.507812,22.75,147.007812
1,"140.0, 162.0078125","28.0, 18.5078125",140.00,162.007812,28.00,18.507812
2,"151.25, 20.5078125","24.75, 128.0078125",151.25,20.507812,24.75,128.007812
3,"127.25, 27.5078125","24.75, 137.0078125",127.25,27.507812,24.75,137.007812
4,"36.75, 122.0078125","163.25, 18.5078125",36.75,122.007812,163.25,18.507812


In [30]:
sorted([3,5,1])

[1, 3, 5]

플레이어 0, 1의 스타팅 x, y 좌표 다 가져와서 sorting (unique하게 만들기 위해)

In [34]:
df_train['both_starting'] = df_train.apply(lambda x: str(sorted([x.player0_x,x.player0_y,x.player1_x,x.player1_y])), axis='columns')

In [35]:
df_train.head()

,player0_starting,player1_starting,player0_x,player0_y,player1_x,player1_y,both_starting
game_id,,,,,,,
0,"145.25, 21.5078125","22.75, 147.0078125",145.25,21.507812,22.75,147.007812,"[21.5078125, 22.75, 145.25, 147.0078125]"
1,"140.0, 162.0078125","28.0, 18.5078125",140.00,162.007812,28.00,18.507812,"[18.5078125, 28.0, 140.0, 162.0078125]"
2,"151.25, 20.5078125","24.75, 128.0078125",151.25,20.507812,24.75,128.007812,"[20.5078125, 24.75, 128.0078125, 151.25]"
3,"127.25, 27.5078125","24.75, 137.0078125",127.25,27.507812,24.75,137.007812,"[24.75, 27.5078125, 127.25, 137.0078125]"
4,"36.75, 122.0078125","163.25, 18.5078125",36.75,122.007812,163.25,18.507812,"[18.5078125, 36.75, 122.0078125, 163.25]"


In [37]:
# df_train.both_starting.value_counts().head(50)